# **2 深度可分离卷积**

## **2.1 什么是深度可分离卷积**

深度可分离卷积的思想来源于**Inception V3**，但是分离是建立在通道上的而不是卷积核上

<div align='center'>
<img src='../image/depthwisecnn.png'>
<div>

上图将通道分为三份,对每一份都使用$3 \times 3$的卷积核分别处理

- 计算量较普通卷积小很多
- 便于在小型设备运行

## **2.2 加载数据集**

In [1]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os, sys, time
import tensorflow as tf
from tensorflow import keras

fashion_mnist = keras.datasets.fashion_mnist
(X_train_all, y_train_all), (X_test, y_test) = fashion_mnist.load_data()

X_valid, X_train = X_train_all[:5000], X_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]
print(X_train.shape, y_train.shape, X_valid.shape, y_valid.shape)

# (x - mean) / std
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
std_scaler.fit(X_train.astype(np.float32).reshape(-1, 1))
X_train_scaled = std_scaler.transform(X_train.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28, 1)
X_valid_scaled = std_scaler.transform(X_valid.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28, 1)

(55000, 28, 28) (55000,) (5000, 28, 28) (5000,)


## **2.3模型搭建**

In [2]:
# tf.keras.models.Sequential
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(filters=32, kernel_size=3, padding='same',
                              activation='selu', input_shape = [28, 28, 1]))
model.add(keras.layers.SeparableConv2D(filters=32, kernel_size=3, padding='same',
                              activation='selu'))
model.add(keras.layers.MaxPool2D(pool_size=2))
model.add(keras.layers.SeparableConv2D(filters=64, kernel_size=3, padding='same',
                              activation='selu')) # 为了缓解信息损失给filters翻倍
model.add(keras.layers.SeparableConv2D(filters=64, kernel_size=3, padding='same',
                              activation='selu'))
model.add(keras.layers.MaxPool2D(pool_size=2))
model.add(keras.layers.SeparableConv2D(filters=128, kernel_size=3, padding='same',
                              activation='selu'))
model.add(keras.layers.SeparableConv2D(filters=128, kernel_size=3, padding='same',
                              activation='selu'))
model.add(keras.layers.MaxPool2D(pool_size=2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))
# Sequential可以接受一个*args参数
# compile
model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [3]:
# fit
history = model.fit(X_train_scaled, y_train, epochs=10, validation_data=(X_valid_scaled, y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 19s 343us/sample - loss: 0.5231 - accuracy: 0.8059 - val_loss: 0.3841 - val_accuracy: 0.8614
Epoch 2/10
55000/55000 [==============================] - 16s 290us/sample - loss: 0.3051 - accuracy: 0.8881 - val_loss: 0.2838 - val_accuracy: 0.8932
Epoch 3/10
55000/55000 [==============================] - 16s 290us/sample - loss: 0.2579 - accuracy: 0.9045 - val_loss: 0.2452 - val_accuracy: 0.9052
Epoch 4/10
55000/55000 [==============================] - 16s 291us/sample - loss: 0.2251 - accuracy: 0.9167 - val_loss: 0.2417 - val_accuracy: 0.9062
Epoch 5/10
55000/55000 [==============================] - 17s 301us/sample - loss: 0.1987 - accuracy: 0.9267 - val_loss: 0.2524 - val_accuracy: 0.9088
Epoch 6/10
55000/55000 [==============================] - 17s 308us/sample - loss: 0.1776 - accuracy: 0.9333 - val_loss: 0.2471 - val_accuracy: 0.9116
Epoch 7/10
55000/55000 [=====================